In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
from tensorflow.python import keras
import numpy as np


In [12]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

data = pd.read_csv('/Users/wenjunshi/Desktop/spam.csv', encoding='latin-1')

In [13]:
y = list(data['v1'])
x = list(data['v2'])

le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [37]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
test = encode(le, ['ham', 'spam', 'ham', 'ham'])
print(test)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [34]:
test = encode(le, ['ham', 'spam', 'ham', 'ham'])

untest = decode(le, test)

x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])
print(len(y_enc))

5572


In [21]:
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K


In [23]:
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=32)

y_test = decode(le, y_test)
y_preds = decode(le, predicts)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1022 13:15:13.807011 4605568448 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


Epoch 1/1
5000/5000 [==============================] - 1276s 255ms/step - loss: 0.0910 - acc: 0.9712


In [24]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

[[497   1]
 [ 11  63]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       498
        spam       0.98      0.85      0.91        74

    accuracy                           0.98       572
   macro avg       0.98      0.92      0.95       572
weighted avg       0.98      0.98      0.98       572

